In [2]:
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.utils import upload_to_neuroglancer as upload
from brainlit.utils import upload_skeleton
from cloudvolume import Bbox
import numpy as np
from pathlib import Path
import napari

## Uploading Brain Images from data in the Octree format.
This notebook demonstrates uploading the 2 lowest-resolution brain volumes, as well as a `.swc` segment file.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `p` is the prefix string. Filepaths take `file://`, while it may also be `s3://` or `gc://`
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, a filepath).

In [3]:
p = "file://"
source = str(Path().resolve().parents[2] / "tests" / "data_octree")
source_segments = str(Path().resolve().parents[2] / "tests" / "data_swcs")
dest = str(Path() / "upload")
dest_segments = str(Path() / "upload_segments")

## 2) Gather information from the octree data.
Data on filepaths, binarized paths, voxel size, and image dimensions are collected.

In [4]:
files, bin_paths, vox_size, tiff_dims = upload.get_volume_info(source, 2, channel=0)
print("Low res files: " + str(files[0]))
print("\nHigh res files: " + str(files[1]))
print("---")
print("Single image binary: " + str(bin_paths[0]))
print("\nMultiple image binaries: " + str(bin_paths[1]))

got files and binary representations of paths.
got dimensions of volume
Low res files: ['/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/default.0.tif']

High res files: ['/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/7/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/6/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/1/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/8/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/4/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/3/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/2/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/5/default.0.tif']
---
Single image binary: [[]]

Multiple image binaries: [['110'], ['101'], ['

## 3) Genrate an `info` file and generate a Cloudvolume `volume` instance.

In [5]:
vols = upload.create_image_layer(p+dest, tiff_dims, vox_size, 2)
print("Number of volumes: " + str(len(vols)))
print("mips: " + str(vols[0].mip) + ' and ' + str(vols[1].mip))
print("Volumes info: " + str(vols[0].info))
print("---")
print("High res volume info: " + str(vols[0].info['scales'][0]))
print("\nLow res volume info: " + str(vols[1].info['scales'][1]))

Number of volumes: 2
mips: 1 and 0
Volumes info: {'data_type': 'uint16', 'num_channels': 1, 'scales': [{'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '9560_9741_31628', 'resolution': [9560, 9741, 31628], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}, {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '19120_19482_63256', 'resolution': [19120, 19482, 63256], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}], 'type': 'image'}
---
High res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '9560_9741_31628', 'resolution': [9560, 9741, 31628], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}

Low res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '19120_19482_63256', 'resolution': [19120, 19482, 63256], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}


## 4) Upload the image data.
This can be done with either Joblib parallel or non-parrallel sequential.

In [6]:
%%capture
u1=upload.upload_chunks(vols[0], files[0], bin_paths[0], parallel=False) # Low res
u2=upload.upload_chunks(vols[1], files[1], bin_paths[1], parallel=False) # High res

## Steps 2-4 are done for the segment data below.
Note that we are using a different package and slightly different methods.

In [7]:
# 2 Get info from source directory
origin, _, _ = upload_skeleton.get_volume_info(source, 2) 
# 3 Create segment info file and volume object
vol = upload_skeleton.create_skeleton_layer(p+source_segments, vox_size, tiff_dims, 2)
# 4 upload the data
skeletons, segids = upload_skeleton.create_skel_segids(p+dest_segments, origin)
for skel in skeletons:
    vol.skeleton.upload(skel)

converting swcs to neuroglancer format...: 0it [00:00, ?it/s]


## Visualize your data with NeuroglancerSession and napari

In [8]:
ngl_sess = NeuroglancerSession(mip=1, url=p+dest, url_segments=p+dest_segments)
bounds = Bbox((0,0,0), tiff_dims)
img = ngl_sess.pull_bounds_img(bounds)
G_sub = ngl_sess.get_segments(2, bounds)
paths = graph_to_paths(G_sub)

Downloading: 100%|██████████| 256/256 [00:00<00:00, 300.11it/s]


NameError: name 'graph_to_paths' is not defined

In [ ]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)